In [ ]:
from multiprocessing import context
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Settings, Document
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.google_genai import GoogleGenAIEmbedding
from llama_index.llms.google_genai import GoogleGenAI
from llama_index.core.prompts import RichPromptTemplate
from dotenv import load_dotenv
import os

load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

Settings.api_key = GOOGLE_API_KEY
Settings.embed_model = GoogleGenAIEmbedding(model_name="gemini-embedding-001")
Settings.llm = GoogleGenAI(
    model="gemini-2.5-flash",
    temperature=0.0
)

# Define the custom system prompt
custom_prompt = """
Sua tarefa é responder a perguntas sobre as estratégias de trading com base no contexto fornecido.
Para cada resposta, forneça a galeria de fotos (galeryId) relevante no formato JSON.
Não inclua markdown.
Se a galeria não for mencionada no contexto, responda apenas com a informação textual.
Formato de saída:
[{
  "textBlock": "Sua resposta textual aqui.",
  "galeryId": <o ID da galeria relevante ou null>
},{
  "textBlock": "Sua resposta textual aqui.",
  "galeryId": <o ID da galeria relevante ou null>
},{
  "textBlock": "Sua resposta textual aqui.",
  "galeryId": <o ID da galeria relevante ou null>
},]

Context information is below.
---------------------
{{context_str}}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {{query_str}}
Answer: 
"""

strategy_text = """
# Opening Range Breakout PlayBook

**Regras da estratégia**

**Entrada:**

1. Depois que fechar fora do range
2. O range começa quando a maxima da barra anterior for maior que a barra atual. Como na barra 1. Ela marca o inicio do range. A barra 3 forma o segundo ponto do range, o de venda.
3. Veja que a barra 2 não fechou fora do range portanto não é uma entrada
4. A barra 4 fechou fora, então é uma entrada.

**Variaçoes:**

Se a primeria barra for bem grande e forte. Como a barra 1

**Saida:**

Stop AMA12

A barra 1 foi a segunda barra fechando dentro da média. Quando a segunda barra fechar dentro da média, Você coloca o stop nesta segunda barra.

Se for o terceiro fechamento dentro

No caso da imagem onde o preço entra na media duas vezes muito próximas, mantenha o stop no ponto 1

No caso das imagens acima nós seriamos estopados na região da barra 1, mas poderiamos entrar novamente na barra 3. Lembrando que a operação começou muito antes, então a contagem de barras dentro da média ja estaria adiantada.

Outros exemplos:

Exemplos:

Entrada no fechamento da barra 1. Na barra 2 esta operação ainda não está no ponto de encurtar o stop.

Eu encurtei o stop prematuramente para o ponto do triangulo verde. Fui stopado sem necessidade.

Somente apos o fechamento da barra 3 seria bom encurtar o stop.

O rompimento pode começar na primeira barra. Se ela for grande 350~400+ pontos

Esse foi um stop correto. Na barra 2

"""


data = {
    "name": "Opening Range Breakout",
    "description": "Trade de rompimento de range de abertura",
    "content": [
        [{"text":"1. Depois que fechar fora do range", "galeryId": 1},
        {"text":"2. O range começa quando a maxima da barra anterior for maior que a barra atual. Como na barra 1. Ela marca o inicio do range. A barra 3 forma o segundo ponto do range, o de venda.", "galeryId": 2},
        {"text":"3. Veja que a barra 2 não fechou fora do range portanto não é uma entrada", "galeryId": None},
        {"text":"4. A barra 4 fechou fora, então é uma entrada.", "galeryId": None}]
    ]
}

# strategy_text = " ".join(
#     f'{item["text"]} {{"galeryId":{item["galeryId"] if item["galeryId"] is not None else "null"}}}'
#     for sublist in data["content"] for item in sublist
# )

# Create a Document object
strategy_document = Document(
    text=strategy_text,
    metadata={
        "name": data["name"],
        "description": data["description"]
    }
)

# Create a VectorStoreIndex from the documents
index = VectorStoreIndex.from_documents([strategy_document], transformations=[SentenceSplitter(chunk_size=200, chunk_overlap=5)])

# Use the custom system prompt when creating the query engine
qa_prompt_tmpl = RichPromptTemplate(custom_prompt)
query_engine = index.as_query_engine(text_qa_template=qa_prompt_tmpl)
response = query_engine.query("Me fale sobre possiveis problemas que posso encontrar no trade")


print(response)

[{
  "textBlock": "Um problema que pode ser encontrado é encurtar o stop prematuramente, o que pode levar a ser stopado sem necessidade.",
  "galeryId": null
},{
  "textBlock": "A operação pode não estar no ponto de encurtar o stop, e fazê-lo antes do momento certo (como antes do fechamento da barra 3) pode ser um erro.",
  "galeryId": null
},{
  "textBlock": "Também é possível ser stopado em certas regiões, como na região da barra 1, embora o contexto mencione a possibilidade de reentrar na barra 3.",
  "galeryId": null
}]


In [ ]:
print(len(response.source_nodes))

for node_with_score in response.source_nodes:
    print(node_with_score)

2
Node ID: 92cdabd8-ff54-4de3-a75c-b12a77ef68ce
Text: Outros exemplos:  Exemplos:  Entrada no fechamento da barra 1.
Na barra 2 esta operação ainda não está no ponto de encurtar o stop.
Eu encurtei o stop prematuramente para o ponto do triangulo verde. Fui
stopado sem necessidade.  Somente apos o fechamento da barra 3 seria
bom encurtar o stop.  O rompimento pode começar na primeira barra. Se
ela ...
Score:  0.712

Node ID: 916c0e09-1bdb-407b-a42d-15d6594f45af
Text: Como a barra 1  **Saida:**  Stop AMA12  A barra 1 foi a segunda
barra fechando dentro da média. Quando a segunda barra fechar dentro
da média, Você coloca o stop nesta segunda barra.  Se for o terceiro
fechamento dentro  No caso da imagem onde o preço entra na media duas
vezes muito próximas, mantenha o stop no ponto 1  No caso das imagens
acima...
Score:  0.706



In [8]:

from IPython.display import Markdown, display
query_engine = index.as_query_engine()
def display_prompt_dict(prompts_dict):
    for k, p in prompts_dict.items():
        text_md = f"**Prompt Key**: {k}<br>" f"**Text:** <br>"
        display(Markdown(text_md))
        print(p.get_template())
        display(Markdown("<br><br>"))
display_prompt_dict(query_engine.get_prompts())

**Prompt Key**: response_synthesizer:text_qa_template<br>**Text:** <br>

Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {query_str}
Answer: 


<br><br>

**Prompt Key**: response_synthesizer:refine_template<br>**Text:** <br>

The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Refined Answer: 


<br><br>

In [1]:
data = {
    "name": "Opening Range Breakout",
    "description": "Trade de rompimento de range de abertura",
    "content": [
        [{"text":"1. Depois que fechar fora do range", "galeryId": 1},
        {"text":"2. O range começa quando a maxima da barra anterior for maior que a barra atual. Como na barra 1. Ela marca o inicio do range. A barra 3 forma o segundo ponto do range, o de venda.", "galeryId": 2},
        {"text":"3. Veja que a barra 2 não fechou fora do range portanto não é uma entrada", "galeryId": None},
        {"text":"4. A barra 4 fechou fora, então é uma entrada.", "galeryId": None}]
    ]
}

strategy_text = " ".join(
    f'{item["text"]} {{"galeryId":{item["galeryId"] if item["galeryId"] is not None else "null"}}}'
    for sublist in data["content"] for item in sublist
)

strategy_text

'1. Depois que fechar fora do range {"galeryId":1} 2. O range começa quando a maxima da barra anterior for maior que a barra atual. Como na barra 1. Ela marca o inicio do range. A barra 3 forma o segundo ponto do range, o de venda. {"galeryId":2} 3. Veja que a barra 2 não fechou fora do range portanto não é uma entrada {"galeryId":null} 4. A barra 4 fechou fora, então é uma entrada. {"galeryId":null}'

In [38]:
from llama_index.core import VectorStoreIndex, Settings, get_response_synthesizer, PromptTemplate
from llama_index.embeddings.google_genai import GoogleGenAIEmbedding
from llama_index.llms.google_genai import GoogleGenAI
from llama_index.core.schema import TextNode
from llama_index.core.prompts import RichPromptTemplate
from llama_index.core.retrievers import VectorIndexRetriever
from dotenv import load_dotenv
import os

load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

Settings.api_key = GOOGLE_API_KEY
Settings.embed_model = GoogleGenAIEmbedding(model_name="gemini-embedding-001")
Settings.llm = GoogleGenAI(
    model="gemini-2.5-flash",
    temperature=0.0
)

custom_prompt ="""
Sua tarefa é responder a perguntas sobre as estratégias de trading com base no contexto fornecido.
Para cada resposta, forneça a galeria de fotos (galeryId) relevante no formato JSON.
Não inclua markdown.
Se a galeria não for mencionada no contexto, responda apenas com a informação textual.
Formato de saída:
[{
  "textBlock": "Sua resposta textual aqui.",
  "galeryId": <o ID da galeria relevante ou null>
},{
  "textBlock": "Sua resposta textual aqui.",
  "galeryId": <o ID da galeria relevante ou null>
},{
  "textBlock": "Sua resposta textual aqui.",
  "galeryId": <o ID da galeria relevante ou null>
},]

Context information is below.
---------------------
{{context_str}}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {{query_str}}
Answer: 
"""

# 1. Create a list of TextNode objects
my_nodes = [
    TextNode(
        text="1. Depois que fechar fora do range",
        metadata={"source": "AI_history_book", "galeryId": "s98fys9dfys9df"}
    ),
    TextNode(
        text="2. O range começa quando a maxima da barra anterior for maior que a barra atual. Como na barra 1. Ela marca o inicio do range. A barra 3 forma o segundo ponto do range, o de venda.",
        metadata={"source": "LLM_research_paper", "author": "Dr. Smith"}
    ),
    TextNode(
        text="3. Veja que a barra 2 não fechou fora do range portanto não é uma entrada",
        metadata={"source": "ethics_journal", "year": 2024, "galeryId": "s98fys9dfys9df"}
    ),
    TextNode(
        text="4. A barra 4 fechou fora, então é uma entrada.",
        metadata={"source": "ethics_journal", "year": 2024, "galeryId": "s98fys9dfys9df"}
    ),
    TextNode(
        text="**Variaçoes:**",
        metadata={"source": "ethics_journal", "year": 2024, "galeryId": "s98fys9dfys9df"}
    ),
    TextNode(
        text="Se a primeria barra for bem grande e forte. Como a barra 1",
        metadata={"source": "ethics_journal", "year": 2024, "galeryId": "s98fys9dfys9df"}
    ),
    TextNode(
        text="Stop AMA12",
        metadata={"source": "ethics_journal", "year": 2024, "galeryId": "s98fys9dfys9df"}
    ),
    TextNode(
        text="Stop AMA12",
        metadata={"source": "ethics_journal", "year": 2024, "galeryId": "s98fys9dfys9df"}
    ),
    TextNode(
        text="A barra 1 foi a segunda barra fechando dentro da média. Quando a segunda barra fechar dentro da média, Você coloca o stop nesta segunda barra.",
        metadata={"source": "ethics_journal", "year": 2024, "galeryId": "s98fys9dfys9df"}
    ),
    TextNode(
        text="Se for o terceiro fechamento dentro",
        metadata={"source": "ethics_journal", "year": 2024, "galeryId": "s98fys9dfys9df"}
    ),
    TextNode(
        text="No caso da imagem onde o preço entra na media duas vezes muito próximas, mantenha o stop no ponto 1",
        metadata={"source": "ethics_journal", "year": 2024, "galeryId": "s98fys9dfys9df"}
    ),
    TextNode(
        text="No caso das imagens acima nós seriamos estopados na região da barra 1, mas poderiamos entrar novamente na barra 3. Lembrando que a operação começou muito antes, então a contagem de barras dentro da média ja estaria adiantada.",
        metadata={"source": "ethics_journal", "year": 2024, "galeryId": "s98fys9dfys9df"}
    )
]

nodes_with_gallery = [
    TextNode(
        text="""
# Opening Range Breakout PlayBook

**Regras da estratégia**

**Entrada:**

1. Depois que fechar fora do range
2. O range começa quando a maxima da barra anterior for maior que a barra atual. Como na barra 1. Ela marca o inicio do range. A barra 3 forma o segundo ponto do range, o de venda.
3. Veja que a barra 2 não fechou fora do range portanto não é uma entrada
4. A barra 4 fechou fora, então é uma entrada.

**Variaçoes:**

Se a primeria barra for bem grande e forte. Como a barra 1

**Saida:**

Stop AMA12

A barra 1 foi a segunda barra fechando dentro da média. Quando a segunda barra fechar dentro da média, Você coloca o stop nesta segunda barra.

Se for o terceiro fechamento dentro

No caso da imagem onde o preço entra na media duas vezes muito próximas, mantenha o stop no ponto 1

No caso das imagens acima nós seriamos estopados na região da barra 1, mas poderiamos entrar novamente na barra 3. Lembrando que a operação começou muito antes, então a contagem de barras dentro da média ja estaria adiantada.

Outros exemplos:

Exemplos:

Entrada no fechamento da barra 1. Na barra 2 esta operação ainda não está no ponto de encurtar o stop.

Eu encurtei o stop prematuramente para o ponto do triangulo verde. Fui stopado sem necessidade.

Somente apos o fechamento da barra 3 seria bom encurtar o stop.

O rompimento pode começar na primeira barra. Se ela for grande 350~400+ pontos

Esse foi um stop correto. Na barra 2

        """,
        metadata={"formated": """
<document>
  <section id="introduction">
    <title>Opening Range Breakout PlayBook</title>
    <markdownContent>
      # Opening Range Breakout PlayBook

      **Regras da estratégia**

      **Entrada:**

      <p>1. Depois que fechar fora do range 
      <asset id="8s7d76shsjd7d7d7yshs7d7d" type="galery"/></p>
      2. O range começa quando a maxima da barra anterior for maior que a barra atual. Como na barra 1. Ela marca o inicio do range. A barra 3 forma o segundo ponto do range, o de venda. 
      3. Veja que a barra 2 não fechou fora do range portanto não é uma entrada 
      <asset id="8s7d766shsjd7dd7yshs7d7d" type="galery"/>
      4. A barra 4 fechou fora, então é uma entrada.

      **Variaçoes:**

      <p>Se a primeria barra for bem grande e forte. Como a barra 1 
      <asset id="8s7d766shsjd7dd7yshs7d7d" type="galery"/><p>

      **Saida:**

      Stop AMA12

      A barra 1 foi a segunda barra fechando dentro da média. Quando a segunda barra fechar dentro da média, Você coloca o stop nesta segunda barra.
      <asset id="8s7d766shsj7d7d7yshs7d7d" type="galery"/>
      Se for o terceiro fechamento dentro 
    </markdownContent>
  </section>
</document>
        """
        
        
        },
        # excluded_embed_metadata_keys=["formated"],
        excluded_llm_metadata_keys=["formated"]
    )
    
]

# 2. Build an index from your custom nodes
# You can use any type of index (e.g., VectorStoreIndex, KeywordTableIndex)
# The index will automatically handle tokenization and embedding of your nodes.
index = VectorStoreIndex(nodes=nodes_with_gallery)

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=5,
)

qa_prompt_tmpl = RichPromptTemplate(custom_prompt)
query_engine = index.as_query_engine(text_qa_template=qa_prompt_tmpl)
# response = query_engine.query("Baseado no texto quais são as variações?")

response_synthesizer = get_response_synthesizer(
    text_qa_template=qa_prompt_tmpl,
)


nodes = retriever.retrieve("Quais são as variaçoes?")
for node in nodes:
    # print(node.)
    # node.node.text = node.metadata["formated"]
    print(node.score)

# response = response_synthesizer.synthesize("Quais são as variaçoes?", nodes=nodes)
# Print the response and the metadata of the retrieved source nodes
# print(response)
# print("\n--- Retrieved Source Node Metadata ---")
# for node in response.source_nodes:
#     print(node)

0.6661171328406459
